- Training code is https://www.kaggle.com/sishihara/ump-lgbm-train-and-save/, but it doesn't work in kaggle code due to ROM.
- I used ForestInference for faster prediction, inspired by the Shopee 2nd place solution. https://www.kaggle.com/lyakaap/2nd-place-solution 

In [ ]:
import joblib
from cuml import ForestInference
import treelite


models = [
    joblib.load('../input/umplgbmmodels/lgb_0.pkl'),
    joblib.load('../input/umplgbmmodels/lgb_1.pkl'),
    joblib.load('../input/umplgbmmodels/lgb_2.pkl'),
    joblib.load('../input/umplgbmmodels/lgb_3.pkl'),
    joblib.load('../input/umplgbmmodels/lgb_4.pkl')
]

gpu_models = []
for clf in models:
    clf.save_model('/tmp/tmp.lgb')
    fi = ForestInference()
    fi.load_from_treelite_model(treelite.Model.load('/tmp/tmp.lgb', model_format='lightgbm'))
    gpu_models.append(fi)

In [ ]:
import ubiquant
import cupy as cp
import numpy as np


env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.drop(["row_id"], axis=1)
    X_test = cp.asarray(X_test.values.astype(np.float32))
    sample_prediction_df["target"] = np.mean([clf.predict(X_test).get() for clf in gpu_models], axis=0)
    env.predict(sample_prediction_df)